In [13]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/My Drive/pSp_self_customize')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import dlib
import glob

import cv2
import numpy as np

from tqdm import tqdm
from PIL import Image

In [15]:
##############################################################################
"""
test_input = "image_datasets/test_images"
test_original_path = "image_datasets/test_resized"
test_processed_path = "image_datasets/test_mosaiced"

os.makedirs(test_original_path, exist_ok=True)
os.makedirs(test_processed_path, exist_ok=True)
"""
##############################################################################
"""
input_path = "image_datasets/ffhq5k_6k_related/ffhq5k_6k_aligned"
output_ori_path = "image_datasets/ffhq5k_6k_related/ffhq5k_6k_mosaiced_neighbor09/ffhq5k_6k_mosaiced_ori"
output_pro_path = "image_datasets/ffhq5k_6k_related/ffhq5k_6k_mosaiced_neighbor09/ffhq5k_6k_mosaiced"
"""
##############################################################################
#100, 80, 13, 09, 07, 06, 05, 04, 03
input_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
output_ori_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor05/ffhq10k_11k_mosaiced_ori"
output_pro_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor05/ffhq10k_11k_mosaiced"

os.makedirs(output_ori_path, exist_ok=True)
os.makedirs(output_pro_path, exist_ok=True)


In [16]:
#Load face landmark detection
detector = dlib.get_frontal_face_detector()


In [17]:
def mosaic(img, face_rectangle, neighbor):
  x1 = face_rectangle.left()
  y1 = face_rectangle.top()
  w = face_rectangle.right() - face_rectangle.left()
  h = face_rectangle.bottom() - face_rectangle.top()

  for i in range(0, h, neighbor): # 从0到h，步长为neighbor
    for j in range(0, w, neighbor):
      rect = [x1+j, y1+i]
      color = img[y1+i, x1+j].tolist()
      left_up = (rect[0], rect[1])
      x2 = rect[0] + neighbor - 1
      y2 = rect[1] + neighbor - 1

      if x2 > x1+w:
        x2 = x1+w
      if y2 > y1+h:
        y2 = y1+h
      right_down = (x2,y2)
      cv2.rectangle(img, pt1=left_up, pt2=right_down, color=color, thickness=-1)

  return img


In [18]:
counts = 0
file_paths = glob.glob(os.path.join(input_path,"*.png"))
try:
  with tqdm(file_paths, total=len(file_paths)) as t:
    for f in t:
      image_np = dlib.load_rgb_image(f)
      dets = detector(image_np,1)

      image_cv = cv2.imread(f)
      try:
        mosaic_img = mosaic(img=image_cv.copy(), face_rectangle=dets[0], neighbor=5)
        f_name = f.split("/")[-1]

        image_cv = cv2.resize(image_cv,(256,256))
        mosaic_img = cv2.resize(mosaic_img,(256,256))
        
        Image.fromarray(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)).save(os.path.join(output_ori_path,f_name))
        Image.fromarray(cv2.cvtColor(mosaic_img, cv2.COLOR_BGR2RGB)).save(os.path.join(output_pro_path,"mosaiced_"+f_name))
      except:
        continue
      counts += 1
except KeyboardInterrupt:
  t.close()
  raise

print("\n{} images have been mosaiced!".format(counts))


100%|██████████| 998/998 [02:15<00:00,  7.36it/s]


988 images have been mosaiced!
